<a href="https://colab.research.google.com/github/ArMOUNO/deep_learning_models/blob/Flowers-recognition-and-Helth-prediction/ensemble_5model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://ieeexplore.ieee.org/abstract/document/9033031
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = drive.ListFile({'q':"title='flower.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('flower.zip')
f.keys()
!unzip flower.zip

Streaming output truncated to the last 5000 lines.
  inflating: flower/0015/10078.jpg   
  inflating: flower/0015/1008.jpg    
  inflating: flower/0015/10083.jpg   
  inflating: flower/0015/10086.jpg   
  inflating: flower/0015/10087.jpg   
  inflating: flower/0015/1009.jpg    
  inflating: flower/0015/10092.jpg   
  inflating: flower/0015/10093.jpg   
  inflating: flower/0015/10095.jpg   
  inflating: flower/0015/10097.jpg   
  inflating: flower/0015/10100.jpg   
  inflating: flower/0015/10101.jpg   
  inflating: flower/0015/10102.jpg   
  inflating: flower/0015/10105.jpg   
  inflating: flower/0015/10110.jpg   
  inflating: flower/0015/10111.jpg   
  inflating: flower/0015/10112.jpg   
  inflating: flower/0015/10114.jpg   
  inflating: flower/0015/10118.jpg   
  inflating: flower/0015/10120.jpg   
  inflating: flower/0015/10122.jpg   
  inflating: flower/0015/10123.jpg   
  inflating: flower/0015/10124.jpg   
  inflating: flower/0015/10138.jpg   
  inflating: flower/0015/10139.jpg   

In [ ]:
gpus = tf.config.list_logical_devices('GPU')
stg=tf.distribute.MirroredStrategy(gpus)

In [ ]:
import cv2
import os
from tqdm import tqdm

folder_dir = '/content/flower'
SIZE = 224
DOWNSAMPLE_RATIO = 4
JPEG_QUALITY = 100

total_files = sum(len(files) for _, _, files in os.walk(folder_dir))

with tqdm(total=total_files, desc="Processing Images") as pbar:
    for folder in os.listdir(folder_dir):
        for file in os.listdir(os.path.join(folder_dir, folder)):
                image_path = os.path.join(folder_dir, folder, file)
                img = cv2.imread(image_path)
                img_resized = cv2.resize(img, (SIZE,SIZE))
                cv2.imwrite(image_path, img_resized)
                pbar.update(1)

Processing Images: 100%|██████████| 14400/14400 [00:40<00:00, 358.91it/s]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

2.13.0


In [ ]:
batch_size = 32
picture_size = (224, 224)
train_set = tf.keras.utils.image_dataset_from_directory(
    directory=folder_dir,
    shuffle=True,
    image_size=picture_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed = 22
)

validation_set = tf.keras.utils.image_dataset_from_directory(
    directory=folder_dir,
    shuffle=True,
    image_size=picture_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed = 22
)


Found 14400 files belonging to 24 classes.
Using 11520 files for training.
Found 14400 files belonging to 24 classes.
Using 2880 files for validation.


In [ ]:
base_model = keras.applications.MobileNet (
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.
base_model_1 = keras.applications.ResNet50V2 (
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.
base_model_2 = keras.applications.VGG16 (
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.
base_model_3 = keras.applications.DenseNet201 (
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.
base_model_4 = keras.applications.ConvNeXtTiny (
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomTranslation, RandomFlip, RandomZoom
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import shutil
import random
import matplotlib.pyplot as plt
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

no_of_classes = 24

# Define the Xception base model
#base_model = MobileNet(weights="imagenet", include_top=False, input_shape=(SIZE, SIZE, 3))
#base_model_1 = ResNet50V2(weights="imagenet", include_top=False, input_shape=(SIZE, SIZE, 3))
#base_model_2 = VGG16(weights="imagenet", include_top=False, input_shape=(SIZE, SIZE, 3))
#base_model_3 = DenseNet201(weights="imagenet", include_top=False, input_shape=(SIZE, SIZE, 3))
#base_model_4 = ConvNeXtTiny(weights="imagenet", include_top=False, input_shape=(SIZE, SIZE, 3))

# Create three instances of the base model with different inputs
input_1 = base_model_1.input
output_1 = base_model_1.output

input_2 = base_model.input
output_2 = base_model.output

input_3 = base_model_2.input
output_3 = base_model_2.output

input_4 = base_model_3.input
output_4 = base_model_3.output


input_5 = base_model_4.input
output_5 = base_model_4.output

# Add global average pooling layer and dense layers for each model
output_1 = GlobalAveragePooling2D()(output_1)
output_1 = Dense(512, activation='relu')(output_1)
output_1 = Dropout(0.5)(output_1)
output_1 = Dense(32, activation='relu')(output_1)
output_1 = Dense(no_of_classes, activation='softmax')(output_1)

output_2 = GlobalAveragePooling2D()(output_2)
output_2 = Dense(512, activation='relu')(output_2)
output_2 = Dropout(0.5)(output_2)
output_2 = Dense(32, activation='relu')(output_2)
output_2 = Dense(no_of_classes, activation='softmax')(output_2)

output_3 = GlobalAveragePooling2D()(output_3)
output_3 = Dense(512, activation='relu')(output_3)
output_3 = Dropout(0.5)(output_3)
output_3 = Dense(32, activation='relu')(output_3)
output_3 = Dense(no_of_classes, activation='softmax')(output_3)

output_4 = GlobalAveragePooling2D()(output_4)
output_4 = Dense(512, activation='relu')(output_4)
output_4 = Dropout(0.5)(output_4)
output_4 = Dense(32, activation='relu')(output_4)
output_4 = Dense(no_of_classes, activation='softmax')(output_4)


output_5 = GlobalAveragePooling2D()(output_5)
output_5 = Dense(512, activation='relu')(output_5)
output_5 = Dropout(0.5)(output_5)
output_5 = Dense(32, activation='relu')(output_5)
output_5 = Dense(no_of_classes, activation='softmax')(output_5)

# Create three separate models
model_1 = Model(inputs=input_1, outputs=output_1)
model_2 = Model(inputs=input_2, outputs=output_2)
model_3 = Model(inputs=input_3, outputs=output_3)
model_4 = Model(inputs=input_4, outputs=output_4)
model_5 = Model(inputs=input_5, outputs=output_5)

# Compile the models
model_1.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model_2.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model_3.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model_4.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model_5.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])




In [ ]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.1,
                                 patience=2,
                                 verbose=1)
early_stopping = EarlyStopping(monitor='val_loss',
                                patience=2,
                                verbose=1)

callbacks = [lr_scheduler, early_stopping]

In [ ]:
# Train model 1
model_1.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                    steps_per_epoch=len(train_set), validation_steps=len(validation_set))

# Train model 2
model_2.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                    steps_per_epoch=len(train_set), validation_steps=len(validation_set))

# Train model 3
model_3.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                    steps_per_epoch=len(train_set), validation_steps=len(validation_set))

# Train model 4
model_4.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                    steps_per_epoch=len(train_set), validation_steps=len(validation_set))

# Train model 5
model_5.fit(train_set, epochs=10, validation_data=validation_set, callbacks=callbacks,
                    steps_per_epoch=len(train_set), validation_steps=len(validation_set))



Epoch 1/10
360/360 [==============================] - 157s 305ms/step - loss: 1.3012 - accuracy: 0.6044 - val_loss: 2.9456 - val_accuracy: 0.4833 - lr: 0.0010
Epoch 2/10
360/360 [==============================] - 108s 301ms/step - loss: 0.4620 - accuracy: 0.8562 - val_loss: 1.5072 - val_accuracy: 0.6955 - lr: 0.0010
Epoch 3/10
360/360 [==============================] - 108s 301ms/step - loss: 0.2963 - accuracy: 0.9043 - val_loss: 0.2347 - val_accuracy: 0.9260 - lr: 0.0010
Epoch 4/10
360/360 [==============================] - 108s 301ms/step - loss: 0.2079 - accuracy: 0.9352 - val_loss: 0.7403 - val_accuracy: 0.8101 - lr: 0.0010
Epoch 5/10
360/360 [==============================] - ETA: 0s - loss: 0.1735 - accuracy: 0.9484
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
360/360 [==============================] - 108s 300ms/step - loss: 0.1735 - accuracy: 0.9484 - val_loss: 0.3194 - val_accuracy: 0.9205 - lr: 0.0010
Epoch 5: early stopping
Epoch 1/10
360/360 

In [ ]:
ensemble_predictions = []
num_models = 5
test_data_dir = '/content/flower'
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=picture_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)
# Generate predictions for each model and collect them in a list
for model in [model_1, model_2, model_3,model_4,model_5]:
    predictions = model.predict_generator(test_generator)
    ensemble_predictions.append(predictions)

# Compute the average predictions of the ensemble
ensemble_predictions = np.mean(ensemble_predictions, axis=0)

# Calculate the accuracy of the ensemble predictions
ensemble_acc = np.mean(np.argmax(ensemble_predictions, axis=1) == test_generator.classes)

print('Ensemble accuracy:', ensemble_acc)


Found 14400 images belonging to 24 classes.


<ipython-input-17-c4371e5900a0>:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_generator)


Ensemble accuracy: 0.9895833333333334
